In [ ]:
import os
import re
import pandas as pd

## INSERT DATA PATH HERE
data_path = ""
column_headers = ['Product', 'CO2 from raw materials', 'CO2 from energy supply', 'N2O & CH4 from production', 'CO2 from production', 'CO2 captured in product', 'Total (incl. captured CO2)', 'Total(excl. captured CO2)']

In [ ]:
def read_txt_to_pd(text:str, column_headers) -> pd.DataFrame:
    start = 0
    while start < len(text):
        first = text[start:].find('0.')
        name = text[start:start+first]
        last_search = re.search('[a-zA-Z]', text[start+first:])
        try: last = last_search.span()[0]
        except: last = len(text)-start+first
        values = text[start+first:start+first+last-1].rstrip().replace(" ", ",")
        with open('temp.txt', 'a') as f:
            f.write(name.replace(',','').rstrip()+','+values+'\n')
        start = start+first+last

    emission_factors = pd.read_csv('temp.txt', header=None)
    os.remove('temp.txt')
    emission_factors.columns = column_headers
    return emission_factors

In [ ]:
# Single value read
f = open(data_path+'all_fertilisers.txt', "r")
text = f.read().replace("\n", " ")
emission_factors = read_txt_to_pd(text, column_headers)

regional = pd.DataFrame()
for file in os.listdir(data_path+'emission_factor_txts/'):
    f = open(data_path+'emission_factor_txts/'+file)
    text = f.read().replace("\n", " ")
    fert = read_txt_to_pd(text, column_headers).rename(columns={'PRODUCT':'Region'})
    fert['PRODUCT'] = file.split('.tx')[0]
    regional = pd.concat((regional, fert), axis=0)

In [ ]:
converted = emission_factors.copy()
converted['ifa_CO2e_100a_Direct Process factor'] = -converted['CO2 captured in product']
converted['ifa_CO2e_100a_Direct Utilities factor'] = converted['N2O & CH4 from production']+converted['CO2 from production']
converted = converted.rename(columns={'CO2 from raw materials':'ifa_CO2e_100a_Raw Material factor', 'CO2 from energy supply':'ifa_CO2e_100a_Indirect Utilities factor', 'Total(excl. captured CO2)':'ifa_CO2e_100a_CtOG factor'}).drop(columns=['Total (incl. captured CO2)', 'CO2 captured in product', 'CO2 from production', 'N2O & CH4 from production'])
converted['ifa_CO2e_100a_CtOG factor'] = converted['ifa_CO2e_100a_CtOG factor']+converted['ifa_CO2e_100a_Direct Process factor']

converted

In [ ]:
converted.to_csv('../data/extracted/conversionFactors_from_IFA.csv')

In [ ]:
## Add uncertainties

converted = pd.read_csv('../data/extracted/conversionFactors_from_IFA.csv', index_col=0)

conversion_uncertainty = 0.1

for col in converted.columns[1:]:
    converted[col.replace('factor','sigma')] = converted[col]*conversion_uncertainty

converted.to_csv('../data/extracted/conversionFactors_from_IFA_w_uncertainties.csv')